# create_retriever_tool 
retriever 를 도구로 변환합니다.

기본 retriever 세팅

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 파일 열기
with open("./data/의대증원반대.txt", "r", encoding="utf-8") as file:
    # 파일 내용 읽기
    content = file.read()


# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_texts = text_splitter.split_text(content)

# VectorStore를 생성합니다.
vector = FAISS.from_texts(split_texts, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever를 도구로 변환

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="text_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the text document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

tools = [retriever_tool]

agent 생성

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [11]:
agent_executor.invoke({"input" : "의사가 전문 뇌약품으로 처방하는 나라는 많아?"})

{'input': '의사가 전문 뇌약품으로 처방하는 나라는 많아?',
 'output': '전문 뇌약품으로 의사가 처방하는 나라는 거의 없다고 합니다. 특히 한국에서는 뇌영양제와 같은 전문 뇌약품이 의사에 의해 처방되는 경우가 드물며, 전 세계적으로도 효과가 미미하다고 언급되고 있습니다. 이러한 뇌약품에 대한 지출이 한국에서 연간 약 1조 원에 달하는데, 이는 예방이나 불필요한 치료를 줄이는 데 비해 약물 치료에 많은 비용이 들어간다는 점을 지적하고 있습니다.'}